negative-mining.sh

train set

In [ ]:
#!/usr/bin/bash
#SBATCH -J bge-m3
#SBATCH --gres=gpu:1
#SBATCH --cpus-per-gpu=6
#SBATCH --mem-per-gpu=20G
#SBATCH -w aurora-g3
#SBATCH -p batch_ugrad
#SBATCH -t 1-0
#SBATCH -o logs/slurm-%A.out
  
pwd
which python
hostname

python hn_mine.py \
--embedder_name_or_path BAAI/bge-m3 \
--input_file /data2/local_datasets/bge-m3/ft_data/relevant_incidents.jsonl \
--output_file /data2/local_datasets/bge-m3/ft_data/relevant_incidents_minedHN.jsonl \
--range_for_sampling 2-300 \
--negative_number 32 \
--use_gpu_for_searching 

exit 0

test (or validation) set

In [ ]:
#!/usr/bin/bash
#SBATCH -J bge-m3
#SBATCH --gres=gpu:1
#SBATCH --cpus-per-gpu=8
#SBATCH --mem-per-gpu=32G
#SBATCH -w aurora-g8
#SBATCH -p batch_ugrad
#SBATCH -t 1-0
#SBATCH -o logs/slurm-%A.out

pwd

python hn_mine.py \
--embedder_name_or_path BAAI/bge-m3 \
--input_file /data2/local_datasets/bge-m3/data/relevant_incidents_test.jsonl \
--output_file /data2/local_datasets/bge-m3/data/relevant_incidents_test_minedHN.jsonl \
--range_for_sampling 2-300 \
--negative_number 8 \
--use_gpu_for_searching

hn_mine.py

In [ ]:
# copied from https://github.com/FlagOpen/FlagEmbedding/blob/master/scripts/hn_mine.py

# MIT License

# Copyright (c) 2022 staoxiao

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.


import json
import random
import numpy as np
from tqdm import tqdm
from typing import Optional
from dataclasses import dataclass, field

import faiss
from transformers import HfArgumentParser
from FlagEmbedding import FlagAutoModel
from FlagEmbedding.abc.inference import AbsEmbedder


@dataclass
class DataArgs:
    """
    Data arguments for hard negative mining.
    """

    input_file: str = field(
        metadata={"help": "The input file for hard negative mining."}
    )
    output_file: str = field(
        metadata={"help": "The output file for hard negative mining."}
    )
    candidate_pool: Optional[str] = field(
        default=None,
        metadata={
            "help": "The candidate pool for hard negative mining. If provided, it should be a jsonl file, each line is a dict with a key 'text'."
        },
    )
    range_for_sampling: str = field(
        default="10-210", metadata={"help": "The range to sample negatives."}
    )
    negative_number: int = field(
        default=15, metadata={"help": "The number of negatives."}
    )
    use_gpu_for_searching: bool = field(
        default=False, metadata={"help": "Whether to use faiss-gpu for searching."}
    )
    search_batch_size: int = field(
        default=64, metadata={"help": "The batch size for searching."}
    )


@dataclass
class ModelArgs:
    """
    Model arguments for embedder.
    """

    embedder_name_or_path: str = field(
        metadata={"help": "The embedder name or path.", "required": True}
    )
    embedder_model_class: Optional[str] = field(
        default=None,
        metadata={
            "help": "The embedder model class. Available classes: ['encoder-only-base', 'encoder-only-m3', 'decoder-only-base', 'decoder-only-icl']. Default: None. For the custom model, you need to specifiy the model class.",
            "choices": [
                "encoder-only-base",
                "encoder-only-m3",
                "decoder-only-base",
                "decoder-only-icl",
            ],
        },
    )
    normalize_embeddings: bool = field(
        default=True, metadata={"help": "whether to normalize the embeddings"}
    )
    pooling_method: str = field(
        default="cls", metadata={"help": "The pooling method fot the embedder."}
    )
    use_fp16: bool = field(
        default=True, metadata={"help": "whether to use fp16 for inference"}
    )
    devices: Optional[str] = field(
        default=None, metadata={"help": "Devices to use for inference.", "nargs": "+"}
    )
    query_instruction_for_retrieval: Optional[str] = field(
        default=None, metadata={"help": "Instruction for query"}
    )
    query_instruction_format_for_retrieval: str = field(
        default="{}{}", metadata={"help": "Format for query instruction"}
    )
    examples_for_task: Optional[str] = field(
        default=None, metadata={"help": "Examples for task"}
    )
    examples_instruction_format: str = field(
        default="{}{}", metadata={"help": "Format for examples instruction"}
    )
    trust_remote_code: bool = field(
        default=False, metadata={"help": "Trust remote code"}
    )
    cache_dir: str = field(
        default=None, metadata={"help": "Cache directory for models."}
    )
    # ================ for inference ===============
    batch_size: int = field(
        default=3000, metadata={"help": "Batch size for inference."}
    )
    embedder_query_max_length: int = field(
        default=512, metadata={"help": "Max length for query."}
    )
    embedder_passage_max_length: int = field(
        default=512, metadata={"help": "Max length for passage."}
    )

    def __post_init__(self):
        # replace "\\n" with "\n"
        if "\\n" in self.query_instruction_format_for_retrieval:
            self.query_instruction_format_for_retrieval = (
                self.query_instruction_format_for_retrieval.replace("\\n", "\n")
            )
        if "\\n" in self.examples_instruction_format:
            self.examples_instruction_format = self.examples_instruction_format.replace(
                "\\n", "\n"
            )


def create_index(embeddings: np.ndarray, use_gpu: bool = False):
    index = faiss.IndexFlatIP(len(embeddings[0]))
    embeddings = np.asarray(embeddings, dtype=np.float32)
    if use_gpu:
        co = faiss.GpuMultipleClonerOptions()
        co.shard = True
        co.useFloat16 = True
        index = faiss.index_cpu_to_all_gpus(index, co=co)
    index.add(embeddings)
    return index


def batch_search(
    index: faiss.Index, query: np.ndarray, topk: int = 200, batch_size: int = 64
):
    all_scores, all_inxs = [], []
    for start_index in tqdm(
        range(0, len(query), batch_size), desc="Batches", disable=len(query) < 256
    ):
        batch_query = query[start_index : start_index + batch_size]
        batch_scores, batch_inxs = index.search(
            np.asarray(batch_query, dtype=np.float32), k=topk
        )
        all_scores.extend(batch_scores.tolist())
        all_inxs.extend(batch_inxs.tolist())
    return all_scores, all_inxs


def get_corpus(candidate_pool: str):
    corpus = []
    with open(candidate_pool, "r", encoding="utf-8") as f:
        for line in f.readlines():
            line = json.loads(line.strip())
            corpus.append(line["text"])
    return corpus


def find_knn_neg(
    model: AbsEmbedder,
    input_file: str,
    output_file: str,
    candidate_pool: Optional[str] = None,
    sample_range: str = "10-210",
    negative_number: int = 15,
    use_gpu: bool = False,
):
    corpus = []
    queries = []
    train_data = []
    for line in open(input_file):
        line = json.loads(line.strip())
        train_data.append(line)
        corpus.extend(line["pos"])
        if "neg" in line:
            corpus.extend([line["neg"]])
        queries.append(line["query"])

    if candidate_pool is not None:
        if not isinstance(candidate_pool, list):
            candidate_pool = get_corpus(candidate_pool)
        corpus = list(set(candidate_pool))
    else:
        corpus = list(set(corpus))

    print(f"inferencing embedding for corpus (number={len(corpus)})--------------")
    p_vecs = model.encode(corpus)
    print(f"inferencing embedding for queries (number={len(queries)})--------------")
    q_vecs = model.encode_queries(queries)

    # check if the embeddings are in dictionary format: M3Embedder
    if isinstance(p_vecs, dict):
        p_vecs = p_vecs["dense_vecs"]
    if isinstance(q_vecs, dict):
        q_vecs = q_vecs["dense_vecs"]

    print("create index and search------------------")
    index = create_index(p_vecs, use_gpu=use_gpu)
    _, all_inxs = batch_search(index, q_vecs, topk=sample_range[-1])
    assert len(all_inxs) == len(train_data)

    for i, data in enumerate(train_data):
        query = data["query"]
        inxs = all_inxs[i][sample_range[0] : sample_range[1]]
        filtered_inx = []
        for inx in inxs:
            if inx == -1:
                break
            if corpus[inx] not in data["pos"] and corpus[inx] != query:
                filtered_inx.append(inx)

        if len(filtered_inx) > negative_number:
            filtered_inx = random.sample(filtered_inx, negative_number)
        data["neg"] = [corpus[inx] for inx in filtered_inx]

    with open(output_file, "w") as f:
        for data in train_data:
            if len(data["neg"]) < negative_number:
                samples = random.sample(
                    corpus, negative_number - len(data["neg"]) + len(data["pos"])
                )
                samples = [sent for sent in samples if sent not in data["pos"]]
                data["neg"].extend(samples[: negative_number - len(data["neg"])])
            f.write(json.dumps(data, ensure_ascii=False) + "\n")


def load_model(model_args: ModelArgs):
    model = FlagAutoModel.from_finetuned(
        model_name_or_path=model_args.embedder_name_or_path,
        model_class=model_args.embedder_model_class,
        normalize_embeddings=model_args.normalize_embeddings,
        pooling_method=model_args.pooling_method,
        use_fp16=model_args.use_fp16,
        query_instruction_for_retrieval=model_args.query_instruction_for_retrieval,
        query_instruction_format=model_args.query_instruction_format_for_retrieval,
        devices=model_args.devices,
        examples_for_task=model_args.examples_for_task,
        examples_instruction_format=model_args.examples_instruction_format,
        trust_remote_code=model_args.trust_remote_code,
        cache_dir=model_args.cache_dir,
        batch_size=model_args.batch_size,
        query_max_length=model_args.embedder_query_max_length,
        passage_max_length=model_args.embedder_passage_max_length,
    )
    return model


def main(data_args: DataArgs, model_args: ModelArgs):
    model = load_model(model_args)

    find_knn_neg(
        model=model,
        input_file=data_args.input_file,
        output_file=data_args.output_file,
        candidate_pool=data_args.candidate_pool,
        sample_range=[int(x) for x in data_args.range_for_sampling.split("-")],
        negative_number=data_args.negative_number,
        use_gpu=data_args.use_gpu_for_searching,
    )


if __name__ == "__main__":
    parser = HfArgumentParser((DataArgs, ModelArgs))
    data_args, model_args = parser.parse_args_into_dataclasses()
    data_args: DataArgs
    model_args: ModelArgs
    main(data_args, model_args)


In [ ]:
{
    "id": 410,
    "title": "제181조",
    "query": "제181조(과실일수) 과실로 인하여 현주건조물 공용건조물 일반건조물 기차 전차 자동차 선박 항공기 또는 광갱을 침해하여 공공의 위험을 발생하게 한 자는 1천만원 이하의 벌금에 처한다",
    "pos": [
        "피고인은 부주의하게 담배꽁초를 창문 밖으로 던져 아파트 화단에 불이 붙어 입주민들의 대피騒動이 발생한 사실이 있다.",
        "피고인은 공사장에서 안전수칙을 미준수한 채 불꽃작업을 하다가 불씨가 건조물 내로 튀어 집기류에 불이 옮겨붙는 사고로 소방대가 출동했다.",
        "피고인은 차량 정비 작업 시 배터리 탈거 중 스파크로 차량 내부에 불이 번져 인근 주차장의 다른 차량들까지 불이 번질 위험을 초래하였다.",
        "피고인이 항공기 정비 도중 기체 주변에서 인화성 물질을 부주의하게 취급하여 조종실 부근에서 연소가 시작되어 탑승객 승·하차가 통제된 일이 있다.",
    ],
    "neg": [ # 32
        "동거가족에 해당하는 형제의 차량 소유에 대해 피고인이 무단으로 처분하여 권리행사를 방해하였으나, 동거 친족 관계이므로 형이 면제되었다.",
        "피고인은 4명의 공범들과 함께 금속 파이프를 들고 회사 건물로 침입하여 유리문을 부수고 집기를 파손하였다.",
        # ...
    ],
}

{'id': 410,
 'title': '제181조',
 'query': '제181조(과실일수) 과실로 인하여 현주건조물 공용건조물 일반건조물 기차 전차 자동차 선박 항공기 또는 광갱을 침해하여 공공의 위험을 발생하게 한 자는 1천만원 이하의 벌금에 처한다',
 'pos': ['피고인은 부주의하게 담배꽁초를 창문 밖으로 던져 아파트 화단에 불이 붙어 입주민들의 대피騒動이 발생한 사실이 있다.',
  '피고인은 공사장에서 안전수칙을 미준수한 채 불꽃작업을 하다가 불씨가 건조물 내로 튀어 집기류에 불이 옮겨붙는 사고로 소방대가 출동했다.',
  '피고인은 차량 정비 작업 시 배터리 탈거 중 스파크로 차량 내부에 불이 번져 인근 주차장의 다른 차량들까지 불이 번질 위험을 초래하였다.',
  '피고인이 항공기 정비 도중 기체 주변에서 인화성 물질을 부주의하게 취급하여 조종실 부근에서 연소가 시작되어 탑승객 승·하차가 통제된 일이 있다.'],
 'neg': ['동거가족에 해당하는 형제의 차량 소유에 대해 피고인이 무단으로 처분하여 권리행사를 방해하였으나, 동거 친족 관계이므로 형이 면제되었다.',
  '피고인은 4명의 공범들과 함께 금속 파이프를 들고 회사 건물로 침입하여 유리문을 부수고 집기를 파손하였다.',
  '피고인은 자신의 보호를 받는 만 15세의 아동을 위험한 건설 현장에서 잡일을 하도록 건설업자에게 넘겨주었다.',
  '선상 강도단 일원인 피고인은 선장 가족을 위협하여 돈을 뺏고, 선장의 부인을 강간하여 피해자가 정신적·육체적으로 중상을 입게 하였다.',
  '피고인은 경쟁 업체 대표가 불법 영업을 한다는 허위 사실을 지역 신문사에 제보하여 기사화되게 하여 명예를 훼손하였다.',
  '피고인은 도로 위를 달리던 시외버스를 미리 준비한 장애물을 이용해 전복시켜 다수의 승객이 다치게 하였다.',
  '철도 유지보수 업무를 담당하는 직원이 주행 중인 기차의 선로 이상을 발견하고도 즉시 조치하지 않아, 결국 기차가 전복되어 심각한 교통장애를 발생시켰다.',
  '